In [1]:
import lib.custom_callbacks as callbacks
import lib.custom_metrics as metrics
import lib.evaluation as ev
import lib.plotting as plot
import lib.models as models
import lib.dataset as dt

import matplotlib.pyplot as plt
import imgaug.augmenters as iaa
import tensorflow as tf
import pandas as pd
import numpy as np
import datetime
import os

from importlib import reload

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPU


In [3]:
DR_LEVELS_PER_CLASS = [[0], [1,2,3,4]]

IMAGE_SIZE = (540, 540, 3)

# Specify dataset files
TRAIN_FILE = 'DATASET-TRAIN-80.csv'
VALIDATION_FILE = 'DATASET-VALIDATION-10_BALANCED.csv'

ONE_HOT_FORMAT = False

TRAINING_BATCH_SIZE = 6
TRAINING_DATA_AUG = True
TRAINING_BALANCED = False
TRAINING_PREFETCH = 20
TRAINING_TAKE_SIZE = None

VALIDATION_BATCH_SIZE = 12
VALIDATION_DATA_AUG = False
VALIDATION_BALANCED = False
VALIDATION_PREFETCH = 1
VALIDATION_TAKE_SIZE = None

In [4]:
reload(dt)

'''
def create_dataset_new(csv_file, 
                        list_list_classes,
                        balanced=False,
                        apply_data_augmentation=False,
                        batch_size=1, 
                        prefetch_buffer=None, 
                        shuffle=False,
                        size=None):
'''

train_dataset, y_true_train = dt.create_dataset_new(TRAIN_FILE, 
                                                    DR_LEVELS_PER_CLASS, 
                                                    balanced=TRAINING_BALANCED, 
                                                    apply_data_augmentation=TRAINING_DATA_AUG, 
                                                    batch_size=TRAINING_BATCH_SIZE, 
                                                    prefetch_buffer=TRAINING_PREFETCH, 
                                                    one_hot_format=ONE_HOT_FORMAT,
                                                    size=TRAINING_TAKE_SIZE)

val_dataset, y_true_val = dt.create_dataset_new(VALIDATION_FILE, 
                                                DR_LEVELS_PER_CLASS, 
                                                balanced=VALIDATION_BALANCED,
                                                apply_data_augmentation=VALIDATION_DATA_AUG,
                                                batch_size=VALIDATION_BATCH_SIZE,
                                                prefetch_buffer=VALIDATION_PREFETCH, 
                                                is_validation=True,
                                                one_hot_format=ONE_HOT_FORMAT,
                                                size=VALIDATION_TAKE_SIZE)

Shuffle all dataset
Se aplica data aug


## Class weights

In [5]:
from sklearn.utils.class_weight import compute_class_weight

class_weights = compute_class_weight('balanced', classes=np.unique(y_true_train[:,0]), y=y_true_train[:,0])
d_class_weights = dict(enumerate(class_weights))
print(d_class_weights)

{0: 0.6740610690645092, 1: 1.9362775165269543}


In [6]:
# Define model
reload(models)

model = models.resNet50V2.get_model(input_shape=(540,540,3), num_outputs=len(DR_LEVELS_PER_CLASS))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50v2 (Functional)      (None, 2048)              23564800  
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 2)                 4098      
Total params: 23,568,898
Trainable params: 23,523,458
Non-trainable params: 45,440
_________________________________________________________________


In [7]:
# Load weights if they were saved
base_path = 'saved_weights/resNet50V2/SGD_clipn_01/'
save_path = base_path + 'model.h5'

base_path_splitted = base_path.split('/')
for i in range(len(base_path_splitted)):
    path_i = '/'.join(base_path_splitted[:i+1])
    if not os.path.exists(path_i):
        os.mkdir(path_i)

if os.path.exists(base_path + 'best_loss_copy.h5'):
    model.load_weights(base_path + 'best_loss_copy.h5')
    print('Model loaded')

Model loaded


In [8]:
# Save ground truth for validation dataset
np.save(base_path + 'ground_truth_val.npy', y_true_val)

In [9]:
# Compile model
reload(metrics)

classes_names = [''.join(list(map(str,class_))) for class_ in DR_LEVELS_PER_CLASS]

print(classes_names)

metric_AUC_0_1234 = metrics.Wrapper_AUC(classes=[1], original_dr_lvls=classes_names, is_one_hot=ONE_HOT_FORMAT)

metric_Sp_at_95_Sens_0_1234 = metrics.Wrapper_SpecificityAtSensitivity(sensitivity=0.95, classes=[1], original_dr_lvls=classes_names, is_one_hot=ONE_HOT_FORMAT)

['0', '1234']


In [10]:
cbacks = [tf.keras.callbacks.ModelCheckpoint(base_path + 'best' + metric_AUC_0_1234.get_custom_name() + '.h5', 
                                             monitor='val_' + metric_AUC_0_1234.get_custom_name(),
                                             save_best_only=True,
                                             save_weights_only=True,
                                             mode='max'),

          tf.keras.callbacks.ModelCheckpoint(base_path + 'best' + metric_Sp_at_95_Sens_0_1234.get_custom_name() + '.h5', 
                                             monitor='val_' + metric_Sp_at_95_Sens_0_1234.get_custom_name(),
                                             save_best_only=True,
                                             save_weights_only=True,
                                             mode='max'),
          
          tf.keras.callbacks.ModelCheckpoint(base_path + 'best_accuracy.h5',
                                             monitor='val_accuracy',
                                             save_best_only=True,
                                             save_weights_only=True,
                                             mode='max'),

          tf.keras.callbacks.ModelCheckpoint(base_path + 'best_loss.h5',
                                             monitor='val_loss',
                                             save_best_only=True,
                                             save_weights_only=True,
                                             mode='min'),
          
          callbacks.Save_Training_Evolution(base_path + 'training_evolution.csv')
]

# cbacks = [tf.keras.callbacks.ModelCheckpoint(base_path + 'best_loss.h5',
#                                              monitor='val_loss',
#                                              save_best_only=True,
#                                              save_weights_only=True,
#                                              mode='min'),
          
#           callbacks.Save_Training_Evolution(base_path + 'training_evolution.csv')
# ]

In [11]:
reload(metrics)

validation_dir = base_path+'validation_outputs/'

if not os.path.exists(validation_dir):
    os.mkdir(validation_dir)

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer=tf.keras.optimizers.SGD(learning_rate=0.0001, momentum=0.9, clipnorm=0.1), 
              metrics=['accuracy',
                       metric_AUC_0_1234, # DR levels: 0 vs 1,2,3,4
                       metric_Sp_at_95_Sens_0_1234, # DR levels: 0 vs 1,2,3,4
                       metrics.RunningValidation(path=validation_dir, n_columns=len(DR_LEVELS_PER_CLASS)) # THIS METRIC MUST BE INIZIALIZATED BEFORE EVERY TRAINING
                      ]) 

num_epochs = 2000

# history = model.fit(train_dataset.take(50), epochs=num_epochs, validation_data=val_dataset.take(10), verbose=1, callbacks=cbacks, class_weight=d_class_weights)
history = model.fit(train_dataset, epochs=num_epochs, validation_data=val_dataset, verbose=1, callbacks=cbacks, class_weight=d_class_weights)

Epoch 1/2000
9471/9471 [==============================] - 1399s 148ms/step - loss: 0.5867 - accuracy: 0.7712 - AUC_DRlvls_0_1234: 0.7528 - Sp_at_95_sens_DRlvls_0_1234: 0.1688 - RunningValidation: 0.0000e+00 - val_loss: 0.5649 - val_accuracy: 0.7101 - val_AUC_DRlvls_0_1234: 0.7901 - val_Sp_at_95_sens_DRlvls_0_1234: 0.2157 - val_RunningValidation: 1.0000
Epoch 2/2000
9471/9471 [==============================] - 1407s 149ms/step - loss: 0.5803 - accuracy: 0.7777 - AUC_DRlvls_0_1234: 0.7602 - Sp_at_95_sens_DRlvls_0_1234: 0.1637 - RunningValidation: 0.0000e+00 - val_loss: 0.5616 - val_accuracy: 0.7084 - val_AUC_DRlvls_0_1234: 0.7953 - val_Sp_at_95_sens_DRlvls_0_1234: 0.1934 - val_RunningValidation: 1.0000
Epoch 3/2000
9471/9471 [==============================] - 1421s 150ms/step - loss: 0.5766 - accuracy: 0.7843 - AUC_DRlvls_0_1234: 0.7623 - Sp_at_95_sens_DRlvls_0_1234: 0.1697 - RunningValidation: 0.0000e+00 - val_loss: 0.5530 - val_accuracy: 0.7168 - val_AUC_DRlvls_0_1234: 0.8026 - val_Sp_

KeyboardInterrupt: 